<a href="https://colab.research.google.com/github/ZzzTheGamer/XAI/blob/Assignment/Assignment2_interpretable_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

## Load the dataset & Take a look at the data structure

In [ ]:
data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
data.info()

In [ ]:
data.head()

## 1. Data cleaning and preparation


*   Drop 'customerID' column



In [ ]:
data.drop(columns=['customerID'], inplace=True)



*   Convert 'TotalCharges' to numeric, replace missing entries



In [ ]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data.dropna(subset=['TotalCharges'], inplace=True)



*   Convert target variable 'Churn' to numeric (0 for No, 1 for Yes)



In [ ]:
data['Churn'] = data['Churn'].map({'No': 0, 'Yes': 1})



*   Encode of categorical variables



In [ ]:
categorical_columns = data.select_dtypes(include=['object']).columns
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)



*   Standardize numeric features



In [ ]:
# Identify numeric features for scaling
numeric_features = data.select_dtypes(include=[np.number]).columns.drop(['Churn','SeniorCitizen'])

# Apply StandardScaler to numeric features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[numeric_features])

# Replace original numeric features with scaled values
features_scaled = pd.DataFrame(scaled_features, columns=numeric_features, index=data.index)



*   Do outlier influence test
*   If cook's distance exceeds the threshold, then it is a potential high impact observation point




In [ ]:
# Define the independent variables and target
X = data[numeric_features]
y = data['Churn']

# Add a constant for the intercept
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

In [ ]:
# Calculate Cook's Distance
influence = model.get_influence()
cooks_d = influence.cooks_distance[0]

# Visualize Cook's Distance
import matplotlib.pyplot as plt

plt.stem(range(len(cooks_d)), cooks_d, markerfmt=",")
plt.axhline(4 / len(X), color='r', linestyle='--', label='Threshold (4/n)')
plt.title("Cook's Distance")
plt.xlabel("Observation Index")
plt.ylabel("Cook's Distance")
plt.legend()
plt.show()

In [ ]:
influential_points = np.where(cooks_d > (4 / len(X)))[0]
print(f"High impact point index: {influential_points}")



*   However, we will not take any action here because when we look back at the dataset, we find that the numeric data includes tenure and total/monthly charges. For wealthy individuals, their charges can be very high, and such extreme values are meaningful. Ignoring them would risk excluding the wealthy group. Additionally, we have already scaled the data, which helps mitigate the impact of outliers.





*   Combine scaled features with encoded categorical variables and the target variable



In [ ]:
encoded_columns = data.drop(columns=numeric_features).columns
data_scaled = pd.concat([features_scaled, data[encoded_columns]], axis=1)

In [ ]:
data_scaled.info()

In [ ]:
data_scaled.head()

In [ ]:
data_scaled.describe()

In [ ]:
data_scaled.to_csv('data_scaled_churn.csv', index=False)

## 2. Data exploration

In [ ]:
# Ensure binary variables are numeric for correlation computation
data_encoded = data_scaled.copy()
data_encoded = data_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))

# Calculate the correlation matrix
correlation_matrix = data_encoded.corr()

# Extract correlations with 'Churn' and sort them
churn_correlation = correlation_matrix['Churn'].sort_values(ascending=False)

In [ ]:
# Plot the correlation matrix as a heatmap
plt.figure(figsize=(10, 8))
plt.barh(churn_correlation.index, churn_correlation.values)
plt.title('Correlation of Variables with Churn')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Variables')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()



####   Here we can see how factors may influence the churn rate
1.   Negatively correlated variables
*   Tenure: Customers who have been with the company longer are less likely to churn, indicating strong customer loyalty.
*   Contract_Two year and Contract_One year: Customers with longer-term contracts tend to have lower churn rates.
*   TotalCharges: Higher total spending is associated with lower churn, possibly reflecting greater customer loyalty.
*   OnlineSecurity_Yes and TechSupport_Yes: Subscribing to additional security services or technical support reduces the likelihood of churn.
2.   Positively correlated variables
*   PaymentMethod_Electronic check: Customers paying via electronic check have higher churn rates, potentially due to convenience.
*   InternetService_Fiber optic: Customers using fiber optic internet services exhibit higher churn rates, possibly due to higher costs.
*   MonthlyCharges: Higher monthly charges are linked to increased churn, indicating price-sensitive customers are more likely to leave.
*   SeniorCitizen: Older customers show slightly higher churn rates, potentially due to challenges in using technology.








In [ ]:
# Plot the full correlation matrix as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', cbar=True, square=True, linewidths=0.5)
plt.title('Correlation Matrix Heatmap', fontsize=16)
plt.show()

*    From the overall correlation chart, we can validate the statements above. At the same time, we noticed some ***high correlations*** between non-target variables, which could potentially lead to ***multicollinearity***.


*   Do Vif to test multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Prepare data for VIF calculation (ensure all variables are numeric)
X = data_encoded.drop(columns=['Churn'])

# Function to calculate VIF
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = dataframe.columns
    vif_data['VIF'] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data
vif_data = calculate_vif(X)
vif_data

* From the VIF chart, we observe that MonthlyCharges (866.089640) has a very high VIF, which is likely due to strong multicollinearity with TotalCharges.
* Additionally, variables such as InternetService_No, OnlineSecurity_No internet service, and OnlineBackup_No internet service show infinite VIF values, which could be influenced by the shared factor of not having internet service, indicating strong multicollinearity.
* Similarly, PhoneService_Yes also has a high VIF, suggesting potential multicollinearity with MultipleLines_Yes phone service.
* Based on these observations, we will retain one variable from each category while removing other highly collinear variables. We will then recalculate VIF to check if it meets the standard.

In [ ]:
# Function to calculate VIF
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = dataframe.columns
    vif_data['VIF'] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data

# Step 1: Drop `MonthlyCharges`
data_reduced = data_encoded.drop(columns=['MonthlyCharges', 'PhoneService_Yes', 'Churn','OnlineBackup_No internet service',
                      'DeviceProtection_No internet service','TechSupport_No internet service',
                      'TechSupport_No internet service','StreamingMovies_No internet service',
                      'OnlineSecurity_No internet service','StreamingTV_No internet service'], errors='ignore')
# Step 2: Recalculate VIF
vif_result = calculate_vif(data_reduced)
vif_result

* Here we can see all values of vif meet the standard

## 3. Linear Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

* Split the dataset

In [ ]:
# Define the target variable and features
X = data_reduced
y = data_encoded['Churn']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

* Define evaluation function for linear model

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"\n{model_name}:")
    print(f"MSE: {mse:.2f}")
    print(f"R2 Score: {r2:.2f}")

    for feature, coef in zip(X_train.columns, model.coef_):
      print(f"{feature}: {coef:.4f}")

    return model, y_pred

* Here we test three linear models: Ordinary linear regression, ridge model, lasso model
* For ridge regression: It uses L2 regularization to shrink coefficients, addressing multicollinearity and overfitting while retaining all features.
* For lasso regression: It uses L1 regularization to shrink insignificant feature coefficients to zero, enabling feature selection and reducing overfitting.

In [ ]:
# Fit models
lr_model, lr_pred = evaluate_model(LinearRegression(), X_train, X_test, y_train, y_test, "Linear Regression")
ridge_model, ridge_pred = evaluate_model(Ridge(alpha=1.0), X_train, X_test, y_train, y_test, "Ridge Regression")
lasso_model, lasso_pred = evaluate_model(Lasso(alpha=0.01), X_train, X_test, y_train, y_test, "Lasso Regression")

1. From the summary output, we can see the linear regression model, with an MSE of 0.15 and an R² score of 0.25, explains only 25% of the variance in the target variable, indicating limited predictive power.
2. Key insights include that longer tenure and long-term contracts (e.g., one-year and two-year contracts) reduce churn likelihood, while features like fiber-optic internet service and electronic check payments are positively associated with churn. For example, a one-unit increase in tenure decreases churn likelihood by 0.0459, while having InternetService_Fiber optic increases it by 0.1821.

* Here, we defined a function to **check the assumptions** of a linear model, which includes the following assumptions: **Linearity, Homoscedasticity, Normality of Residuals, and Independence**. **Multicollinearity and outliers** have already been detected and corrected earlier, so they are not checked again here.

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from statsmodels.api import OLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import linear_reset

In [ ]:
def check_assumptions(model, X_train, y_train, model_name):
    # Predict on the training set
    y_train_pred = model.predict(X_train)
    residuals = y_train - y_train_pred

    # Linearity: residuals vs. predicted values plot
    plt.figure(figsize=(8, 6))
    plt.scatter(y_train_pred, residuals, alpha=0.5)
    plt.axhline(0, color="red", linestyle="--")
    plt.title(f"Residuals vs. Predicted Values ({model_name})")
    plt.xlabel("Predicted Values")
    plt.ylabel("Residuals")
    plt.show()

    # Homoscedasticity: Breusch-Pagan test
    X_train_with_const = sm.add_constant(X_train)
    bp_test = het_breuschpagan(residuals, X_train_with_const)
    bp_test_result = {
        "Lagrange Multiplier Statistic": bp_test[0],
        "p-value": bp_test[1],
        "Homoscedasticity": "Pass" if bp_test[1] > 0.05 else "Fail",
    }

    # Normality of residuals: Shapiro-Wilk test
    shapiro_test_stat, shapiro_p_value = shapiro(residuals)
    shapiro_test_result = {
        "Test Statistic": shapiro_test_stat,
        "P-Value": shapiro_p_value,
        "Normality": "Pass" if shapiro_p_value > 0.05 else "Fail",
    }

    # Durbin-Watson rest for independence
    durbin_watson_stat = durbin_watson(residuals)

    # Display results
    assumption_checks = {
        "Breusch-Pagan Test (Homoscedasticity)": bp_test_result,
        "Shapiro-Wilk Test (Normality)": shapiro_test_result,
        "Durbin-Watson Test (Independence)": durbin_watson_stat
    }

    return assumption_checks

# Check assumptions for all three models
lr_assumptions = check_assumptions(lr_model, X_train, y_train, "Linear Regression")
ridge_assumptions = check_assumptions(ridge_model, X_train, y_train, "Ridge Regression")
lasso_assumptions = check_assumptions(lasso_model, X_train, y_train, "Lasso Regression")

lr_assumptions, ridge_assumptions, lasso_assumptions

* Here, we can see the residual plot shows a clear stratified pattern, indicating that the linearity assumption may not hold. It may due to the binary nature of the target variable or a nonlinear relationship.
* The normality and homoscedasticity tests fail, while the independence test holds as the value of the DW test is close to 2.

* Below are my solutions. Since the target variable is binary (0 or 1, representing churn or no churn), applying a log transformation to it is neither reasonable nor meaningful. Therefore, I chose to transform the explanatory variables (independent variables). Specifically, I added interaction terms and square terms. These transformations can help address the issues with certain assumptions not being satisfied, such as **linearity**, **homoscedasticity**(nonlinear transformation can reduce the structured pattern of residuals and improve heteroscedasticity), and potentially improving the normality of residuals (the bias can be indirectly reduced, thus increasing the degree of normality satisfaction).

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Generate interaction terms (degree=2 includes original features and pairwise interactions)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_interaction = poly.fit_transform(X_train)
X_test_interaction = poly.transform(X_test)

# Convert the interaction item to a DataFrame
interaction_feature_names = poly.get_feature_names_out(X_train.columns)
X_train_interaction = pd.DataFrame(X_train_interaction, columns=interaction_feature_names, index=X_train.index)
X_test_interaction = pd.DataFrame(X_test_interaction, columns=interaction_feature_names, index=X_test.index)

In [ ]:
lr_model_interaction, lr_pred_interaction = evaluate_model(LinearRegression(), X_train_interaction, X_test_interaction, y_train, y_test, "Linear Regression with Interaction Terms")
ridge_model_interaction, ridge_pred_interaction = evaluate_model(Ridge(alpha=1.0), X_train_interaction, X_test_interaction, y_train, y_test, "Ridge Regression with Interaction Terms")
lasso_model_interaction, lasso_pred_interaction = evaluate_model(Lasso(alpha=0.01), X_train_interaction, X_test_interaction, y_train, y_test, "Lasso Regression with Interaction Terms")

In [ ]:
# Convert transformed data to DataFrame with consistent feature names
X_train_poly = pd.DataFrame(X_train_interaction, columns=interaction_feature_names, index=X_train.index)
X_test_poly = pd.DataFrame(X_test_interaction, columns=interaction_feature_names, index=X_test.index)

# Pass these updated DataFrames to the assumption-checking function
lr_assumptions = check_assumptions(lr_model_interaction, X_train_poly, y_train, "Linear Regression")
ridge_assumptions = check_assumptions(ridge_model_interaction, X_train_poly, y_train, "Ridge Regression")
lasso_assumptions = check_assumptions(lasso_model_interaction, X_train_poly, y_train, "Lasso Regression")

lr_assumptions, ridge_assumptions, lasso_assumptions

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Generate polynomial features (degree=2 includes original and squared terms)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Convert the polynomial features to a DataFrame
poly_feature_names = poly.get_feature_names_out(X_train.columns)
X_train_poly = pd.DataFrame(X_train_poly, columns=poly_feature_names, index=X_train.index)
X_test_poly = pd.DataFrame(X_test_poly, columns=poly_feature_names, index=X_test.index)

# Evaluate models with polynomial features
lr_model_poly, lr_pred_poly = evaluate_model(LinearRegression(), X_train_poly, X_test_poly, y_train, y_test, "Linear Regression with Polynomial Features")
ridge_model_poly, ridge_pred_poly = evaluate_model(Ridge(alpha=1.0), X_train_poly, X_test_poly, y_train, y_test, "Ridge Regression with Polynomial Features")
lasso_model_poly, lasso_pred_poly = evaluate_model(Lasso(alpha=0.01), X_train_poly, X_test_poly, y_train, y_test, "Lasso Regression with Polynomial Features")

In [ ]:
# Pass these updated DataFrames to the assumption-checking function
lr_assumptions = check_assumptions(lr_model_poly, X_train_poly, y_train, "Linear Regression")
ridge_assumptions = check_assumptions(ridge_model_poly, X_train_poly, y_train, "Ridge Regression")
lasso_assumptions = check_assumptions(lasso_model_poly, X_train_poly, y_train, "Lasso Regression")

lr_assumptions, ridge_assumptions, lasso_assumptions

* From this, we can see that even after adding interaction terms and square terms, the performance of the linear model has not significantly improved, and the assumptions are still not satisfied.
* To avoid collinearity and improve interpretation, we choose to proceed with the initial model for interpretation.

In [ ]:
# Coefficient Comparison Plot
def plot_coefficients(lr_model, ridge_model, lasso_model):
    coef_df = pd.DataFrame({
        'Linear': lr_model.coef_,
        'Ridge': ridge_model.coef_,
        'Lasso': lasso_model.coef_
    }, index=X.columns)

    plt.figure(figsize=(12, 6))
    coef_df.plot(kind='bar', width=0.8)
    plt.title('Coefficient Comparison: Linear vs Ridge vs Lasso')
    plt.xlabel('Features')
    plt.ylabel('Coefficient Value')
    plt.legend(loc='best')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

plot_coefficients(lr_model, ridge_model, lasso_model)

* From this graph, we can see that for the linear regression model, certain features (such as tenure, Contract_Two year, and InternetService_Fiber optic) have larger coefficients, indicating their significant impact on the target variable.
* Both Ridge and Lasso retain the importance of these features, but Lasso reduces the coefficients of less significant features (such as PhoneService_Yes and OnlineBackup_Yes) to zero, suggesting these features have lower importance.
* The key features identified as important across all models (Linear, Ridge, and Lasso) are tenure, InternetService_Fiber optic, InternetService_No, Contract_One year, and PaymentMethod_Electronic check. Tenure and Contract_One year act as protective features, reducing churn with their negative coefficients, while Fiber optic and Electronic check increase churn risk with positive coefficients. Additionally, InternetService_No is a significant feature indicating that customers without internet service are less likely to churn. These are consistent with those previously found in the correlation matrix section, where more detailed explanations have been explained without attribution explanations.

In [ ]:
# Feature Importance Heatmap
def plot_feature_importance_heatmap(lr_model, ridge_model, lasso_model):
    importance_df = pd.DataFrame({
        'Linear': np.abs(lr_model.coef_),
        'Ridge': np.abs(ridge_model.coef_),
        'Lasso': np.abs(lasso_model.coef_)
    }, index=X.columns)

    plt.figure(figsize=(12, 8))
    sns.heatmap(importance_df, annot=True, cmap='YlOrRd')
    plt.title('Feature Importance Heatmap')
    plt.tight_layout()
    plt.show()

plot_feature_importance_heatmap(lr_model, ridge_model, lasso_model)

* This heatmap illustrates feature importance across three models (Linear, Ridge, and Lasso). Key features such as **InternetService_Fiber optic, Contract_One year, PaymentMethod_Electronic check, InternetService_No and Tenure** consistently show high importance across all models, indicating their strong influence on churn.
* In contrast, less impactful features such as **gender_Male, Dependents_Yes, and OnlineBackup_Yes** are compressed to zero in Lasso. At the same time, for some relatively important variables of the first two models like **total charges and onlineSecurity_yes**, lasso also penalizes their coefficients, making them less important.
* Ridge does not change much with respect to linear coefficients.

In [ ]:
# Regularization Path for Lasso
from sklearn.linear_model import lasso_path

# Plot Lasso Regularization Path
def plot_lasso_path(X, y, feature_names):
    alphas, coefs, _ = lasso_path(X, y, alphas=np.logspace(-5, 2, 200))
    plt.figure(figsize=(12, 6))
    for coef_path, feature in zip(coefs, feature_names):
        plt.plot(-np.log10(alphas), coef_path, label=feature)
    plt.xlabel('-log(alpha)')
    plt.ylabel('Coefficients')
    plt.title('Lasso Regularization Path')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()

# Call the function to plot
plot_lasso_path(X_train, y_train, X.columns)

* This plot illustrates the Lasso regularization path, showing how feature coefficients change as the regularization strength alpha increases. Key features like **tenure**, **InternetService_Fiber optic**, **Contract_One year**, and **PaymentMethod_Electronic check** retain significant coefficients even under strong regularization, indicating their importance. In contrast, less significant features such as **gender_Male**, **Dependents_Yes**, and **DeviceProtection_Yes** are quickly shrunk to zero.


## Logistic Regression

* Since the target variable is binary and the linear model fails to meet the required assumptions, using a sigmoid model, such as logistic regression, would be a better choice.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import statsmodels.api as sm
from sklearn.metrics import roc_curve, auc
from scipy.stats import shapiro

* Logistic Regression Model

In [ ]:
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)
y_pred_logistic = logistic_model.predict(X_test)

* Here, we define a unified evaluation model for handling classification problems. The evaluation metrics include **accuracy, precision, recall, F1 score, and ROC AUC value.**

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name, threshold=0.5):
    if hasattr(model, 'predict_proba'):
        # Models with probability output
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        # Models with direct prediction output
        y_pred_proba = model.predict(X_test)

    # Convert probabilities to binary predictions
    y_pred = (y_pred_proba >= threshold).astype(int)

    # Evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # ROC and AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc_value = auc(fpr, tpr)

    # Plot ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, lw=2, label=f'{model_name} ROC Curve (AUC = {roc_auc_value:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=1, label='Random Guess')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Return evaluation metrics
    return {
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "ROC AUC": roc_auc_value,
        "Confusion Matrix": conf_matrix.tolist()
    }

In [ ]:
logistic_eval = evaluate_model(logistic_model, X_train, X_test, y_train, y_test, "Logistic Regression")
print(logistic_eval)

1. This ROC curve and evaluation metrics reflect the performance of the logistic regression model on a classification task.
2. The ROC curve shows the model's ability to distinguish between positive and negative classes at different thresholds, with an AUC score of 0.83, indicating good discriminatory power (an 83% probability of correctly distinguishing between positive and negative samples).
3. The model achieves an accuracy of 78.9%, meaning it correctly predicts 78.9% of all samples. However, the precision (62.5%) indicates that only 62.5% of the samples predicted as positive are actual positives, while the recall (51.3%) reveals that the model identifies only 51.3% of all true positives, suggesting room for improvement in detecting positive cases.
4. The F1 score (56.4%) balances precision and recall. The F1 Score is not high, indicating that the model still has room for improvement in the prediction of positive samples.
5. To improve, strategies like adjusting the decision threshold to enhance recall, optimizing feature engineering, balancing the dataset to handle potential class imbalance, or exploring more complex models like random forests or gradient boosting can be considered.

* **Logistic regression assumption checks (influential outliers and multicollinearity were tested earlier)**
* **Test for linearity assumption and converge assumption**

In [ ]:
# Add a log interaction for the Box-Tidwell Test
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

X_train_log = X_train_scaled.copy()

for col in X_train.columns:
    X_train_log[f"{col}_log"] = X_train_scaled[col] * np.log(np.abs(X_train_scaled[col] + 1e-10))

# Fit the model using statsmodels
X_train_log.index = y_train.index
X_train_log = sm.add_constant(X_train_log)
logit_model = sm.Logit(y_train, X_train_log).fit()

# Check the significance level and converge information
print(logit_model.summary())
print(logit_model.mle_retvals)

##### Here we check the linearity assumption:
The linearity assumption in logistic regression suggests that independent variables should have a linear relationship with the log-odds of the dependent variable. To verify this, log interaction terms (e.g., tenure_log, TotalCharges_log) were added, and their p-values were examined. Variables like tenure and TotalCharges had p-values < 0.05 for their log interactions, indicating nonlinearity and the need for transformations. Other variables, such as SeniorCitizen and gender_Male, had non-significant log interactions (p-values of NaN or 1), suggesting they might satisfy the linearity assumption but contribute little to the model. Therefore, transformations are necessary to improve the model fit.

##### We also observe the following issues:
1. Maximum Likelihood optimization failed to converge.
2. Coefficients and standard errors of some variables (e.g., SeniorCitizen) are NaN.
3. warnflag: 1 and converged: False.
4. These indicate that the model failed to converge within the maximum number of iterations, likely due to the following reasons (should check and address further):
Perfect separation: Some variables might perfectly predict the target class, causing coefficient divergence. Insufficient iterations: The default maximum iteration limit may not be adequate for the model to converge High-dimensional features: The number of features relative to the data size may make optimization challenging.

* **Test for perfect separation**

In [ ]:
for col in X_train.columns:
    print(f"{col}: {data_encoded.groupby(col)['Churn'].nunique().max()} unique values in target")

###### The fact that each independent variable (col) corresponds to 2 unique values in the target variable (Churn), rather than 1, indicates that there is no perfect separation.

* **Test for independence of observations**

In [ ]:
# residual calculation
residuals = y_train - logistic_model.predict_proba(X_train)[:, 1]

# Durbin-Watson test
dw_stat = durbin_watson(residuals)
print(f"Durbin-Watson Statistic: {dw_stat}")

###### The Durbin-Watson statistic of 1.995 is very close to 2, which indicates that the residuals of your logistic regression model are independent.

* **Test for large sample size**

In [ ]:
# Check whether the number of positive and negative samples is sufficient
positive_count = sum(y_train == 1)
negative_count = sum(y_train == 0)
num_predictors = X_train.shape[1]

print(f"Positive samples: {positive_count}")
print(f"Negative samples: {negative_count}")
print(f"Required minimum samples per class: {10 * num_predictors}")

###### We can see the current sample distribution (number of positive and negative samples, both of which are smaller than the minumum number required per class) meets the sample size requirements of logistic regression.



* **Test for class imbalance**

In [ ]:
# Calculate class distribution
class_counts = y_train.value_counts()
# Calculate and display proportions
class_proportions = class_counts / len(y_train)
print(class_proportions)

###### This indicates a class imbalance, as the majority class (Non-Churn) accounts for nearly three-fourths of the data, while the minority class (Churn) makes up only about one-fourth.

###### Here we use weighted logistic regression with class_weight='balanced to refine the model. It helps address the issue of imbalanced class distribution by assigning higher weights to the minority class and lower weights to the majority class. While it does not directly resolve assumptions like linearity or independence, it mitigates the perfect separation issue that can arise when the minority class is underrepresented. Additionally, it indirectly improves model stability by making the loss function more sensitive to underrepresented samples, thereby addressing convergence challenges in imbalanced datasets.

In [ ]:
from sklearn.metrics import classification_report
# Use weighted logistic regression
logistic_model_weighted = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, class_weight='balanced')
logistic_model_weighted.fit(X_train, y_train)

# Check weighted model
y_pred_weighted = logistic_model_weighted.predict(X_test)
print("\nConfusion Matrix (Weighted):")
print(confusion_matrix(y_test, y_pred_weighted))
print("\nClassification Report (Weighted):")
print(classification_report(y_test, y_pred_weighted))

1. From the graph, we can see the unweighted logistic regression model achieves higher accuracy (78.9%) and precision (62.5%) compared to the weighted model (73% accuracy and 50% precision). This is because the unweighted model prioritizes the majority negative class (non-churn), resulting in fewer false positives (115 vs. 299).
2. However, the weighted model significantly improves recall for the positive class (churn), increasing it from 51.3% to 79%, which reduces false negatives (182 to 78).
3. The F1 score of the weighted model (61%) is higher than the unweighted model's (56.4%), as it balances the trade-off between precision and recall better for the positive class.
4. While the unweighted model is more precise and overall accurate, the weighted model is better at identifying churned customers, making it suitable when minimizing churn is the priority despite more false alarms.

* **Visualize logistic regression coefficient**

In [ ]:
# Extract feature names and coefficients
coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': logistic_model_weighted.coef_[0]
})

# Sort coefficients by absolute value
coefficients = coefficients.reindex(coefficients['Coefficient'].abs().sort_values(ascending=False).index)

# Plot the coefficients
plt.figure(figsize=(10, 6))
coefficients.plot(kind='barh', x='Feature', y='Coefficient', legend=False, color='skyblue')
plt.axvline(0, color='black', linewidth=0.8)
plt.title('Logistic Regression Coefficients')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

# Print the coefficients
print("Coefficients:")
for feature, coef in zip(X_train.columns, logistic_model_weighted.coef_[0]):
    print(f"{feature}: {coef}")

1. The plot highlights key features influencing churn. Positive coefficients identify factors increasing churn risk ( increase the likelihood of the target being 1), while negative coefficients point to factors reducing churn. For example, TotalCharges, InternetService_Fiber optic, and PaymentMethod_Electronic check have positive coefficients, indicating that these features are associated with a higher probability of churn.  Contract_Two year, tenure, and OnlineSecurity_Yes have negative coefficients, meaning they are associated with a lower likelihood of churn.
2. Features with larger absolute values have the stronger impact on customer behavior. Larger absolute values (e.g., Contract_Two year, tenure) indicate stronger influence on the target. Smaller absolute values (e.g., gender_Male, DeviceProtection_Yes) have minimal impact on predicting churn.
3. In more details, if we use numerical variables interpretation by odds, for example, we could say increasing tenure by 1 unit decreases the odds of churn by a factor of exp(−1.3430)≈0.261, meaning the odds of churn reduce by approximately 73.9% per additional unit of tenure.

## GAM Model

In [ ]:
!pip install pygam

In [ ]:
from pygam import LinearGAM, GammaGAM, s
# Make sure X_train and y_train are NumPy arrays
X_train = X_train.values if hasattr(X_train, 'values') else X_train
y_train = y_train.values if hasattr(y_train, 'values') else y_train

# The formula of dynamically constructing GAM
num_features = X_train.shape[1]
# Initializes the first smoothing term
formula = s(0)
for i in range(1, num_features):
    # Add smooth entries dynamically
    formula += s(i)

# Initialize the GAM model
gam = LinearGAM(formula)

# Grid search smoothing parameters
gam.gridsearch(X_train, y_train)

# Fit model
gam.fit(X_train, y_train)

# Generate prediction
y_pred_gam = gam.predict(X_test)

In [ ]:
gam.summary()

In [ ]:
# See which feature corresponds to each s
feature_names = data_reduced.columns.to_list()
for i, feature in enumerate(feature_names):
    print(f"s({i}) corresponds to feature: {feature}")

1. The results of the Generalized Additive Model (GAM) provide insights into the relationship between each feature and customer churn, which accounts for nonlinear effects.
2. Key significant features (e.g., tenure, TotalCharges, SeniorCitizen, MultipleLines_Yes, InternetService_Fiber optic, InternetService_No, and contract types...) demonstrate strong statistical significance (p-values close to zero), suggesting they are crucial predictors of churn. For instance, tenure has a highly nonlinear relationship with churn, where longer tenures significantly reduce churn risk. Similarly, customers with longer contracts (one-year or two-year) show much lower churn likelihood compared to those without contracts.
3. On the other hand, non-significant features such as gender, whether a customer has dependents, or some payment methods (e.g., credit card automatic payment) exhibit little to no influence on churn, reflecting their limited explanatory power in the model.

*  **Visualize GAM**

###### The below code snippet rows = math.ceil(num_features / 3), plt.figure(figsize=(15, 5 * rows)) were generated using chatgpt4 on 01/17/25 at 8:10pm.

In [ ]:
import math

# Determine the number of rows and columns for subplots
num_features = len(feature_names)
rows = math.ceil(num_features / 3)

plt.figure(figsize=(15, 5 * rows))
subplot_index = 1
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue
    plt.subplot(rows, 3, subplot_index)
    XX = gam.generate_X_grid(term=i)
    plt.plot(XX[:, term.feature], gam.partial_dependence(term=i, X=XX))
    plt.title(feature_names[term.feature])
    plt.ylabel('Partial dependence')
    plt.xlabel(feature_names[term.feature])
    subplot_index += 1
plt.tight_layout()
plt.show()

1. From the graph, we can see each subplot shows how the feature on the x-axis affects the  probability of the target when other features are held constant.
2. For features like tenure and TotalCharges, the curves are smooth, showing the non-linear relationship between the feature and the target. Take Tenure as an example, a non-linear effect is observed where low values contribute positively, but higher values decrease the predicted target probability after a point.
3. For binary features, the graph shows a simple stepwise change between 0 and 1. Take InternetService_Fiber optic as an example, having fiber optic service seems to increase the predicted probability of the target.
4. We can determine the importance of feature according to the value change of the vertical axis. For example, features like tenure, TotalCharges, InternetService_Fiber optic, and Contract_years have substantial impacts on predictions.


In [ ]:
GAM_eval = evaluate_model(gam, X_train, X_test, y_train, y_test, "GAM model")
print(GAM_eval)

###### We can see there are some differencces between Gam and logistic model(weighted)
1. Precision: GAM achieves higher precision for the positive class (0.6518 vs. 0.50), meaning it is more cautious about predicting positive churn, reducing false positives.
2. Recall: Weighted logistic regression excels in recall (0.79 vs. 0.4305), capturing more true positives but at the cost of higher false positives.
3. Accuracy: GAM has a slightly higher overall accuracy (0.7875 vs. 0.73).
4. F1 Score: Weighted logistic regression has a higher F1 score for the positive class (0.61 vs. 0.5185), making it better at balancing precision and recall.

## Comparisons

1. Linear Regression (including Ridge and Lasso):
* Strengths: (1) Predictions are transparent and easy to interpret as coefficients represent the linear relationship between features and the target variable. (2) Lasso regression helps with feature selection by shrinking less important feature coefficients to zero. (3) Ridge and Lasso can also address overfitting by penalizing large coefficients.
* Weaknesses: (1) Linear regression assumes relationships between features and the target are linear, which is often unrealistic in real-world problems like customer churn. In addition, it has to satisfy many conditional assumptions and statistical distribution assumptions, such as Homoscedasticity, Normality etc. (2) It has poor performance for nonlinear data. When relationships are complex (like relationships in this dataset), accuracy decreases. (3) The interpretation of a weight depends on other features being held constant.
2. Logistic Regression (Iincluding Weighted Logistic Regression)
* Strengths: (1) Easy to interpret, though weights are multiplicative (log-odds). (2) The weighted version helps mitigate imbalanced datasets by assigning different penalties to classes. (3) Provides probabilities in addition to classifications, aiding in decision-making. (4) Performs best among the three models for the variable of interest to us (churn=1), with the highest recall rate and f1 score.
* Weakness: (1) Like linear regression, logistic regression is subject to many condition. For example, it assumes linear relationships between features and the log-odds of the target variable. (2) Features that perfectly separate classes can prevent model convergence.
3. GAM (Generalized Additive Model)
* Strengths: (1) Captures nonlinear relationships between features and the target variable, improving predictive performance. (2) Retains partial transparency by allowing visualization of the effect of each feature on the target. (3) Has the highest predictive power due to its highest accuracy of the three models
* Weakness: (1) More computationally intensive and less interpretable than linear models. (2) Requires careful tuning of smoothing parameters to avoid overfitting.

## Recommondations

#### The following models are recommended in order
1. When the priority is to identify as many churners as possible while requiring a certain level of interpretability. Use Weighted Logistic Regression. It handles imbalanced data well and is straightforward to explain to stakeholders. Additionally, it performs well in predicting the variables we are most interested in.
2. For overall predictive power, use GAM. It can model complex, nonlinear relationships, making it more accurate for overall customer churn prediction.
3. Linear regressiona are not recommended as the primary approach due to its inability to handle nonlinear patterns and poor predictive power. However, lasso can be used for feature selection to identify important predictors.
#### In addition to the models already explored, several other options can be applied to the customer churn dataset, depending on priorities such as accuracy, interpretability, and complexity. For predictive performance, Gradient Boosting models like XGBoost or Random Forests are excellent choices, as they handle non-linear relationships effectively. If interpretability is crucial, Decision Trees can also provide clear insights into churn drivers. For smaller datasets, Bayesian Models or SVMs may be useful. Meanwhile, Neural Networks are a powerful option for large datasets. Overall, the choice of the model should align with business goals.